# 🛩️ Building a Historical Flight Tracker with Python and CesiumJS

## Overview and Motivation

This notebook demonstrates how to build an interactive flight history tracking visualization similar to [FlightRadar24](https://www.flightradar24.com/) using real historical [ADS-B (Automatic Dependent Surveillance-Broadcast)](https://en.wikipedia.org/wiki/Automatic_Dependent_Surveillance%E2%80%93Broadcast) flight data, in your notebook.

I created this example and libraries to, next, build a forensic tool for analyzing flight events. The next step will be to add more data sources (e.g. radar, satellite, weather) and advanced analysis features (photos/videos projection: if you have ideas or want to contribute, let me know!)

## What You'll Build

- 🌐 **Fetch live historical data** using [py-adsb-historical-data-client](https://pypi.org/project/py-adsb-historical-data-client/)
- 🗺️ **Create 3D visualizations** with CesiumJS and the cesiumjs_anywidget library
- 🎬 **Build time-dynamic animations** using CZML (Cesium Markup Language)

An interactive 3D globe showing:
- ✈️ Real flight trajectories with animated paths
- 🌍 Time-based filtering (select specific dates/times)
- ⏱️ Timeline controls for playback
- 💾 Efficient local caching to minimize repeated downloads

## What is already built

The core visualization and interaction components are provided by the [cesiumjs_anywidget](https://github.com/Alex-PLACET/cesiumjs_anywidget) library, which wraps [CesiumJS](https://cesium.com/platform/cesiumjs/) for use in Jupyter notebooks.
Today (February 2026), the widget is still in active development and the code is not very clean (like dirty vibe-coded JavaScript). Features available don't yet cover all the functionalities provided by CesiumJS. The current available features are the ones I needed for this demo and other projects, such as:
- CZML data source loading and time-dynamic visualization
- Timeline and clock controls
- Basic camera controls and synchronization with notebook widgets
- Measurement tools
- Data synchronization between Python <=> JavaScript
- ...

## Data Source

Historical ADS-B data is retrieved directly from [ADS-B Exchange](https://globe.adsbexchange.com/) using the [py-adsb-historical-data-client](https://pypi.org/project/py-adsb-historical-data-client/) Python package.

The package provides two levels of data:

### Heatmap data (fast overview)

- Downloaded from `globe.adsbexchange.com/globe_history/{date}/heatmap/{slot}.bin.ttf`
- One file per 30-minute slot, covering all aircraft globally
- Each entry includes ICAO hex code, callsign, position, altitude, and ground speed
- Used for the interactive overview at the bottom of this notebook

### Full trace data (detailed trajectories)

- Downloaded from `globe.adsbexchange.com/globe_history/{date}/traces/{subfolder}/trace_full_{icao}.json`
- One file per aircraft per day; includes a high-resolution time series of positions
- Use the **"Load Full Traces"** button to download detailed paths for visible aircraft

### Local caching

All downloaded files are stored in `./adsb_cache/` so subsequent loads are instant.


## Setup and Imports

In [ ]:
from datetime import datetime
from pathlib import Path
from typing import List, Dict
import random
import os

from cesiumjs_anywidget import CesiumWidget
from py_adsb_historical_data_client import (
    set_cache,
    FullHeatmapEntry,
    TraceEntry,
    get_zoned_heatmap_entries,
)

# Set cache directory next to this notebook
notebook_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
cache_dir = notebook_dir / "adsb_cache"
cache_dir.mkdir(exist_ok=True)

# Configure global cache for persistent local storage
set_cache(cache_dir)

print(f"ADS-B cache directory: {cache_dir}")
print("Data source: globe.adsbexchange.com historical API")


## ⚙️ Performance Configuration

Before loading data, let's configure performance parameters:

### Understanding Performance Trade-offs

When working with large aviation datasets, we need to balance:

- **Data Volume** vs **Load Time**: More flights = richer visualization but slower loading
- **Spatial Coverage** vs **Detail**: Wider area = more context but more data to process
- **Temporal Resolution** vs **Smoothness**: More points per flight = smoother animation but higher memory usage

### Configuration Parameters

In [ ]:
# Performance Configuration
class PerformanceConfig:
    """Configuration for performance optimization.
    
    Adjust these parameters based on your use case:
    - Presentation/Demo: MAX_FLIGHTS=1000, USE_SPATIAL_FILTER=True
    - Fast Exploration: MAX_FLIGHTS=200, RADIUS_MULTIPLIER=0.8
    - Global View: MAX_FLIGHTS=500, USE_SPATIAL_FILTER=False
    """

    # Maximum number of flights to display at once
    # Lower = faster rendering, Higher = more comprehensive view
    MAX_FLIGHTS = 500  # Recommended: 100-1000

    # Whether to use spatial filtering (limit to visible area)
    USE_SPATIAL_FILTER = True  # Set False to see all flights globally

    # Radius multiplier for spatial filtering
    # Higher = larger search area
    RADIUS_MULTIPLIER = 1.0  # Default: 1.0, Larger view: 2.0

    # Minimum points requir0ed for a flight path
    MIN_PATH_POINTS = 2

    # Heatmap visualization mode
    # When True, shows only a trailing tail behind aircraft positions
    USE_HEATMAP_TAILS = True
    HEATMAP_TAIL_SECONDS = 5 * 60  # 5 minutes

    # Full trace visualization mode
    # When True, shows only a trailing tail behind aircraft instead of full-day history
    USE_TRACE_TAILS = True
    TRACE_TAIL_SECONDS = 5 * 60  # 5 minutes

    # Altitude conversion to CZML (Cesium expects WGS84 ellipsoid heights)
    # ADS-B values are often MSL/barometric-like references.
    USE_GEOID_MSL_TO_WGS84 = False  # Enable for more accurate vertical reference
    ALTITUDE_OFFSET_METERS = 0.0    # Optional manual offset if needed

config = PerformanceConfig()
print("⚙️  Performance config loaded:")
print(f"   Max flights: {config.MAX_FLIGHTS}")
print(f"   Spatial filtering: {'Enabled' if config.USE_SPATIAL_FILTER else 'Disabled'}")
print(f"   Radius multiplier: {config.RADIUS_MULTIPLIER}x")
print(f"   Heatmap tails: {'Enabled' if config.USE_HEATMAP_TAILS else 'Disabled'} ({config.HEATMAP_TAIL_SECONDS // 60} min)")
print(f"   Trace tails: {'Enabled' if config.USE_TRACE_TAILS else 'Disabled'} ({config.TRACE_TAIL_SECONDS // 60} min)")
print(f"   Geoid MSL→WGS84: {'Enabled' if config.USE_GEOID_MSL_TO_WGS84 else 'Disabled'}")
print(f"   Manual altitude offset: {config.ALTITUDE_OFFSET_METERS:+.1f} m")

## Helper functions

In [ ]:
def calculate_view_radius(altitude_m: float) -> float:
    """Calculate reasonable view radius based on camera altitude.
    
    Uses a simple heuristic:
    - Low altitude (<10km): ~50km radius
    - Medium altitude (10-100km): ~200km radius
    - High altitude (>100km): ~500km radius
    """
    if altitude_m < 10000:
        return 50000  # 50 km
    elif altitude_m < 100000:
        return 200000  # 200 km
    else:
        return 500000  # 500 km


## CZML Conversion Functions

### Understanding CZML

CZML (Cesium Markup Language) is JSON for describing time-dynamic 3D scenes: https://github.com/AnalyticalGraphicsInc/czml-writer/wiki/CZML-Guid

CZML is a good format to represent time-dynamic entities like aircraft trajectories, as it allows specifying positions, orientations, and properties over time. We will convert our flight data into CZML format for visualization in CesiumJS.

In [ ]:
def generate_random_color() -> List[int]:
    """Generate a random bright color in RGBA format."""
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return [r, g, b, 255]  # Add alpha channel


def _alt_feet_to_meters(alt: int | str | None) -> float:
    """Convert altitude from feet (possibly 'ground') to meters (MSL-like).

    Guards against malformed values (nan/inf/invalid strings).
    """
    import math

    if alt is None or alt == "ground":
        return 100.0

    try:
        altitude_m = float(alt) * 0.3048
    except (ValueError, TypeError):
        return 100.0

    if not math.isfinite(altitude_m):
        return 100.0

    # Keep within a sensible aviation range to avoid viewer artifacts.
    return max(-500.0, min(25000.0, altitude_m))


def _altitude_m_to_czml(altitude_m: float, lat: float, lon: float) -> float:
    """Convert source altitude to CZML height reference.

    Cesium/CZML expects WGS84 ellipsoid heights. Source ADS-B altitude is often
    MSL/barometric-like, so we optionally convert using geoid undulation.
    """
    import math

    adjusted_alt = altitude_m + config.ALTITUDE_OFFSET_METERS

    if config.USE_GEOID_MSL_TO_WGS84:
        try:
            from cesiumjs_anywidget.geoid import msl_to_wgs84
            adjusted_alt = msl_to_wgs84(adjusted_alt, lat, lon)
        except Exception:
            pass

    if not math.isfinite(adjusted_alt):
        return 0
    
    return adjusted_alt


def _to_epoch_sampled_cartographic(
    samples: List[tuple[datetime, float, float, float]],
) -> tuple[str, str, List[float]]:
    """Convert datetime cartographic samples to CZML epoch+offset format."""
    start_dt = samples[0][0]
    end_dt = samples[-1][0]
    epoch_iso = start_dt.isoformat().replace("+00:00", "Z")
    end_iso = end_dt.isoformat().replace("+00:00", "Z")

    sampled: List[float] = []
    for sample_dt, lon, lat, alt in samples:
        seconds = (sample_dt - start_dt).total_seconds()
        sampled.extend([seconds, lon, lat, alt])

    return epoch_iso, end_iso, sampled


def heatmap_entries_to_czml(entries: List[FullHeatmapEntry]) -> List[Dict]:
    """Convert FullHeatmapEntry list to CZML format."""
    czml: List[Dict] = [{"id": "document", "name": "Flight Positions", "version": "1.0"}]

    if not entries:
        print("ℹ️  No heatmap entries to convert to CZML")
        return czml

    flights: Dict[str, List[FullHeatmapEntry]] = {}
    for entry in entries:
        flights.setdefault(entry.hex_id, []).append(entry)

    print(f"Creating CZML for {len(flights)} unique flights...")

    all_timestamps = [e.timestamp for e in entries if e.timestamp is not None]
    if all_timestamps:
        global_start = min(all_timestamps).isoformat().replace("+00:00", "Z")
        global_end = max(all_timestamps).isoformat().replace("+00:00", "Z")
        czml[0]["clock"] = {
            "interval": f"{global_start}/{global_end}",
            "currentTime": global_start,
            "multiplier": 60,
            "range": "LOOP_STOP",
            "step": "SYSTEM_CLOCK_MULTIPLIER",
        }
        print(f"Time range: {global_start} to {global_end}")

    for icao_hex, flight_entries in flights.items():
        color = generate_random_color()
        callsign = flight_entries[0].callsign or icao_hex.upper()

        samples: List[tuple[datetime, float, float, float]] = []
        for e in flight_entries:
            if e.alt == 1635325:
                e.alt = 0 
            sample_dt = e.timestamp  # type: ignore[assignment]
            altitude_m = _altitude_m_to_czml(_alt_feet_to_meters(e.alt), e.lat, e.lon)
            samples.append((sample_dt, e.lon, e.lat, altitude_m))

        start_time, end_time, position_array = _to_epoch_sampled_cartographic(samples)

        entity: Dict = {
            "id": f"flight_{icao_hex}",
            "name": callsign,
            "description": (
                f"<table>"
                f"<tr><td>ICAO:</td><td>{icao_hex.upper()}</td></tr>"
                f"<tr><td>Callsign:</td><td>{callsign}</td></tr>"
                f"<tr><td>Points:</td><td>{len(flight_entries)}</td></tr>"
                f"</table>"
            ),
            "availability": f"{start_time}/{end_time}",
            "position": {"epoch": start_time, "cartographicDegrees": position_array},
            "point": {
                "color": {"rgba": color},
                "pixelSize": 8,
                "outlineColor": {"rgba": [0, 0, 0, 255]},
                "outlineWidth": 2,
            },
        }
        entity["path"] = {
            "material": {
                "polylineOutline": {
                    "color": {"rgba": color},
                    "outlineColor": {"rgba": [0, 0, 0, 100]},
                    "outlineWidth": 1,
                }
            },
            "width": 3,
            "leadTime": 0,
            "trailTime": config.HEATMAP_TAIL_SECONDS,
            "resolution": 5,
        }
        czml.append(entity)

    print(f"✓ Generated CZML with {len(czml) - 1} flight trajectories")
    return czml


def traces_to_czml(traces: Dict[str, List[TraceEntry]]) -> List[Dict]:
    """Convert full trace data to CZML format."""
    czml: List[Dict] = [{"id": "document", "name": "Flight Trajectories", "version": "1.0"}]

    if not traces:
        print("ℹ️  No trace data to convert to CZML")
        return czml

    print(f"Creating CZML for {len(traces)} unique flights from traces...")

    timestamps = (
        entry.timestamp
        for entries in traces.values()
        for entry in entries
        if entry.timestamp is not None
    )

    first_ts = next(timestamps, None)
    min_timestamp: datetime | None = first_ts
    max_timestamp: datetime | None = first_ts

    if first_ts is not None:
        for ts in timestamps:
            if ts < min_timestamp:  # type: ignore[operator]
                min_timestamp = ts
            if ts > max_timestamp:  # type: ignore[operator]
                max_timestamp = ts

    global_interval: str | None = None
    if min_timestamp is not None and max_timestamp is not None:
        global_start = min_timestamp.isoformat().replace("+00:00", "Z")
        global_end = max_timestamp.isoformat().replace("+00:00", "Z")
        global_interval = f"{global_start}/{global_end}"
        czml[0]["clock"] = {
            "interval": global_interval,
            "currentTime": global_start,
            "multiplier": 60,
            "range": "LOOP_STOP",
            "step": "SYSTEM_CLOCK_MULTIPLIER",
        }

    for icao_hex, entries in traces.items():
        color = generate_random_color()
        callsign = icao_hex.upper()
        if entries[0].aircraft:
            flight = entries[0].aircraft.get("flight", "").strip()
            if flight:
                callsign = flight

        samples: List[tuple[datetime, float, float, float]] = []
        for e in entries:
            altitude_msl = (e.altitude or 0) * 0.3048
            altitude_m = _altitude_m_to_czml(altitude_msl, e.latitude, e.longitude)
            samples.append((e.timestamp, e.longitude, e.latitude, altitude_m))

        start_time, end_time, position_array = _to_epoch_sampled_cartographic(samples)

        entity: Dict = {
            "id": f"flight_{icao_hex}",
            "name": callsign,
            "description": (
                f"<table>"
                f"<tr><td>ICAO:</td><td>{icao_hex.upper()}</td></tr>"
                f"<tr><td>Callsign:</td><td>{callsign}</td></tr>"
                f"<tr><td>Points:</td><td>{len(entries)}</td></tr>"
                f"</table>"
            ),
            "availability": global_interval if global_interval is not None else f"{start_time}/{end_time}",
            "position": {"epoch": start_time, "cartographicDegrees": position_array},
            "point": {
                "color": {"rgba": color},
                "pixelSize": 8,
                "outlineColor": {"rgba": [0, 0, 0, 255]},
                "outlineWidth": 2,
            },
        }
        entity["path"] = {
            "material": {
                "polylineOutline": {
                    "color": {"rgba": color},
                    "outlineColor": {"rgba": [0, 0, 0, 100]},
                    "outlineWidth": 1,
                }
            },
            "width": 3,
            "leadTime": 0,
            "trailTime": config.TRACE_TAIL_SECONDS,
            "resolution": 5,
        }

        czml.append(entity)

    print(f"✓ Generated CZML with {len(czml) - 1} flight trajectories")
    return czml


## Flight Data Manager

In [ ]:
class FlightDataManager:
    """Manages flight data loading and updates based on camera position and time.

    Uses py-adsb-historical-data-client to fetch data directly from
    globe.adsbexchange.com with automatic local caching.
    """

    def __init__(self, widget: CesiumWidget, initial_date: datetime):
        self.widget = widget
        self.current_date = initial_date
        self.last_entries: List[FullHeatmapEntry] = []

    def update_data(
        self,
        camera_lat: float,
        camera_lon: float,
        camera_alt: float,
        current_time: datetime | None = None,
    ) -> None:
        """Update flight data based on camera position and time.

        Downloads heatmap data for the given location/time window and
        renders it as CZML in the widget.
        """
        # Default to noon on the current date (timezone-naive for the API)
        if current_time is None:
            current_time = self.current_date.replace(hour=12, minute=0, second=0, microsecond=0)
        # Strip timezone so the API receives a naive datetime
        current_time = current_time.replace(tzinfo=None)

        radius = calculate_view_radius(camera_alt) * config.RADIUS_MULTIPLIER

        print("\nUpdating data...")
        print(f"  Location: ({camera_lat:.2f}, {camera_lon:.2f})")
        print(f"  Altitude: {camera_alt / 1000:.1f} km")
        print(f"  Radius: {radius / 1000:.1f} km")
        print(f"  Time: {current_time}")

        try:
            entries = list(
                get_zoned_heatmap_entries(
                    timestamp=current_time,
                    latitude=camera_lat,
                    longitude=camera_lon,
                    radius=radius,
                )
            )

            unique_icaos = set(e.hex_id for e in entries)
            print(f"  ✓ Found {len(unique_icaos)} unique aircraft ({len(entries)} position entries)")

            # Cap at MAX_FLIGHTS aircraft to keep the viewer responsive
            if len(unique_icaos) > config.MAX_FLIGHTS:
                kept_icaos = set(list(unique_icaos)[: config.MAX_FLIGHTS])
                entries = [e for e in entries if e.hex_id in kept_icaos]
                print(f"  Limiting to {config.MAX_FLIGHTS} aircraft")

            self.last_entries = entries

            if entries:
                czml = heatmap_entries_to_czml(entries)
                print(f"  Loading {len(czml) - 1} flight positions into viewer...")
                self.widget.load_czml(czml)
                print("  ✓ Update complete")
            else:
                print("  No flights found in this area/time")

        except Exception as e:
            print(f"  Error updating data: {e}")
            import traceback

            traceback.print_exc()

    def load_traces(self, icao_list: List[str] | None = None) -> None:
        """Download full flight traces using TraceSession and get_traces."""
        from py_adsb_historical_data_client.historical import TraceSession

        if icao_list is None:
            icao_list = list({e.hex_id for e in self.last_entries})

        if not icao_list:
            print("No ICAO codes to load traces for.")
            return

        print(f"\nDownloading full traces for {len(icao_list)} aircraft...")

        traces: Dict[str, List[TraceEntry]] = {}

        try:
            with TraceSession() as session:
                for icao in icao_list:
                    try:
                        trace_entries = list(session.get_traces(icao, self.current_date))
                    except Exception as exc:
                        print(f"  ✗ {icao.upper()}: {exc}")
                        continue

                    if trace_entries:
                        traces[icao] = trace_entries
        except Exception as exc:
            print(f"  Error creating TraceSession: {exc}")
            return

        print(f"  Trace files successfully parsed: {len(traces)}")

        if traces:
            czml = traces_to_czml(traces)
            entity_count = len(czml) - 1
            print(f"  CZML entities generated: {entity_count}")
            print(f"  Loading {entity_count} full traces into viewer...")
            self.widget.load_czml(czml)
            print("  ✓ Traces loaded")
        else:
            print("  No trace data available")

    def clear_data(self) -> None:
        """Clear all loaded flight data."""
        self.last_entries = []
        self.widget.clear_czml()
        print("Cleared all flight data")

    def change_date(self, new_date: datetime) -> None:
        """Change the active date and reload data."""
        self.current_date = new_date
        self.clear_data()
        print(f"Changed date to {new_date.date()}")

## Initialize Widget

In [ ]:
# Initial date: March 15, 2023 (known to have data)
initial_date = datetime(2023, 3, 15)

# Create widget centered on Paris
widget = CesiumWidget(
    latitude=48.8566,
    longitude=2.3522,
    altitude=50000,  # 50km altitude for good overview
    heading=0,
    pitch=-45,
    roll=0,
    height="800px",
    enable_terrain=False,
    enable_lighting=True,
    show_timeline=True,  # Enable timeline for playback
    animation=True,
    current_time=initial_date.isoformat() + 'Z'
)

print("Widget created. View centered on Paris.")
print(f"Date: {initial_date.date()}")
print("\nControls:")
print("  - Pan/zoom to explore different regions")
print("  - Use timeline to scrub through time")

## Setup Data Manager

In [ ]:
# Create data manager
data_manager = FlightDataManager(widget, initial_date)

## Display widget on a side panel

In [ ]:
from sidecar import Sidecar
sc = Sidecar(title='Flight radar')
with sc:
    display(widget)

## Data Loading

In [ ]:

data_manager.update_data(
    widget.latitude,
    widget.longitude,
    widget.altitude,
    initial_date.replace(hour=15, minute=0)  # 3:00 PM
)

## Interactive Date Selector

Let's create a date picker to select which day's data to load:

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Date picker
date_picker = widgets.DatePicker(
    description='Flight Date:',
    value=initial_date.date(),
    disabled=False
)

# Time slider (hour of day)
time_slider = widgets.IntSlider(
    value=12,
    min=0,
    max=23,
    step=1,
    description='Hour (UTC):',
    continuous_update=False
)

# Buttons
update_button = widgets.Button(
    description='Load Heatmap',
    button_style='primary',
    icon='download'
)

traces_button = widgets.Button(
    description='Load Full Traces',
    button_style='info',
    icon='plane'
)

clear_button = widgets.Button(
    description='Clear All',
    button_style='warning',
    icon='trash'
)

status_label = widgets.Label(value=f'Ready. Current date: {initial_date.date()}')


def on_update_clicked(b):
    new_date = datetime.combine(date_picker.value, datetime.min.time())
    new_time = new_date.replace(hour=time_slider.value)

    if new_date.date() != data_manager.current_date.date():
        data_manager.change_date(new_date)

    status_label.value = f'Loading heatmap for {new_time}...'
    data_manager.update_data(
        widget.latitude,
        widget.longitude,
        widget.altitude,
        new_time
    )
    flight_count = len({e.hex_id for e in data_manager.last_entries})
    status_label.value = f'Loaded {flight_count} aircraft'


def on_traces_clicked(b):
    if not data_manager.last_entries:
        status_label.value = 'Load heatmap first to discover aircraft'
        return
    icao_list = list({e.hex_id for e in data_manager.last_entries})
    status_label.value = f'Downloading traces for {len(icao_list)} aircraft...'
    data_manager.load_traces(icao_list)
    status_label.value = f'Full traces loaded for {len(icao_list)} aircraft'


def on_clear_clicked(b):
    data_manager.clear_data()
    status_label.value = 'Cleared all data'


update_button.on_click(on_update_clicked)
traces_button.on_click(on_traces_clicked)
clear_button.on_click(on_clear_clicked)

controls = widgets.VBox([
    widgets.HBox([date_picker, time_slider]),
    widgets.HBox([update_button, traces_button, clear_button]),
    status_label
])

display(controls)


# Ways to improve

This is a prototype and there are many ways to improve it:
- 🗺️ **Better Spatial filtering**: Allow users to select a geographic region to focus on.
- 🕒 **Temporal filtering**: Allow users to select specific time ranges to focus on (e.g. rush hours, specific events)
- 🛩️ **Flight details**: Add popups with detailed flight information when clicking.
- 🧹 **Data cleanup**: Implement better handling of missing/erroneous data points (e.g. outliers, altitude spikes)
- 🚀 **Performance optimizations**: Use spatial indexing, frustum, downsampling, ...

# About the author

My name is [Alexis Placet](https://www.linkedin.com/in/alexisplacet/) and I'm a software engineer and open-source enthusiast. I'm currently working at [Quantstack](https://www.quantstack.com/) where we build tools for data science and scientific computing.
